In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
import tabulate

In [7]:
df = pd.read_csv('ner.csv', encoding='ISO-8859-1')
df

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...,...,...
47954,Sentence: 47955,Indian border security forces are accusing the...,"['JJ', 'NN', 'NN', 'NNS', 'VBP', 'VBG', 'PRP$'...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'B-gpe..."
47955,Sentence: 47956,Indian officials said no one was injured in Sa...,"['JJ', 'NNS', 'VBD', 'DT', 'NN', 'VBD', 'VBN',...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
47956,Sentence: 47957,Two more landed in fields belonging to a nearb...,"['CD', 'JJR', 'VBD', 'IN', 'NNS', 'VBG', 'TO',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
47957,Sentence: 47958,They say not all of the rockets exploded upon ...,"['PRP', 'VBP', 'RB', 'DT', 'IN', 'DT', 'NNS', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47959 entries, 0 to 47958
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentence #  47959 non-null  object
 1   Sentence    47959 non-null  object
 2   POS         47959 non-null  object
 3   Tag         47959 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [9]:
df.isna().sum()

Sentence #    0
Sentence      0
POS           0
Tag           0
dtype: int64

In [10]:
sentences = df['Sentence'].to_numpy()
tags = df['Tag'].apply(lambda x: x[1:-1].replace("'", "").replace(",", "")).to_numpy()

In [11]:
sentences

array(['Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .',
       'Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "',
       'They marched from the Houses of Parliament to a rally in Hyde Park .',
       ..., 'Two more landed in fields belonging to a nearby village .',
       'They say not all of the rockets exploded upon impact .',
       'Indian forces said they responded to the attack'], dtype=object)

In [12]:
tags

array(['O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O',
       'O O O O O O O O O O O O O O O O O O B-per O O O O O O O O O O O',
       'O O O O O O O O O O O B-geo I-geo O', ...,
       'O O O O O O O O O O O', 'O O O O O O O O O O O',
       'B-gpe O O O O O O O'], dtype=object)

In [13]:
X_train, X_tmp, y_train, y_tmp = train_test_split(
    sentences, tags, test_size=1000, random_state=101)

X_val, X_test, y_val, y_test = train_test_split(
    X_tmp, y_tmp, test_size=0.5, random_state=101)

In [14]:
print(f"Total training samples: {len(X_train)}")
print(f"Total validation samples: {len(X_val)}")
print(f"Total testing samples: {len(X_test)}")

Total training samples: 46959
Total validation samples: 500
Total testing samples: 500


In [15]:
def get_sentence_vectorizer(sentences):
    sentence_vectorizer = layers.TextVectorization(standardize=None)
    sentence_vectorizer.adapt(sentences)
    vocab = sentence_vectorizer.get_vocabulary()

    return sentence_vectorizer, vocab

In [16]:
sentence_vectorizer, vocab = get_sentence_vectorizer(X_train)

In [17]:
print(f"Number of unique words in the training vocab: {len(vocab)}\n")
print(f"Sentence: {X_train[0]}\n")
print(f"Sentence vectorized: {sentence_vectorizer(X_train[0])}")

Number of unique words in the training vocab: 34894

Sentence: Nigeria 's parliament is debating a constitutional amendment proposed by supporters of Mr. Obasanjo that would permit presidents three terms in office instead of two .

Sentence vectorized: [  723    11   273    14 10143     8  1177  3763   944    23   761     5
    34  3601    16    93  4461  2413    94  1892     6   425  1853     5
    44     3]


In [18]:
print(f"Sentence: {X_train[0]}")
print(f"Labels: {y_train[0]}")

Sentence: Nigeria 's parliament is debating a constitutional amendment proposed by supporters of Mr. Obasanjo that would permit presidents three terms in office instead of two .
Labels: B-geo O O O O O O O O O O O B-per B-org O O O O O O O O O O O O


In [19]:
def get_tags(labels):
    tags = set()
    for l in labels:
        for tag in l.split(' '):
            tags.add(tag)

    return list(sorted(tags))

In [20]:
tags = get_tags(y_train)
print(tags)

['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim', 'O']


In [21]:
def label_vectorizer(lables, tags):
    label_ids = []
    for l in lables:
        tags_ids = [tags.index(tag) for tag in l.split(' ')]
        label_ids.append(tags_ids)

    return tf.keras.preprocessing.sequence.pad_sequences(label_ids, padding='post', value=-1)

In [22]:
label_vectorizer([y_train[0], 'O'], tags)

array([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,  6,  5, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [16, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

In [23]:
def generate_dataset(sentences, labels, sentence_vectorizer, label_vectorizer, tags):
    sentences_ids = sentence_vectorizer(sentences)
    label_ids = label_vectorizer(labels, tags)
    ds = tf.data.Dataset.from_tensor_slices((sentences_ids, label_ids))
    return ds

In [24]:
ds_train = generate_dataset(X_train, y_train, sentence_vectorizer, label_vectorizer, tags)
ds_val = generate_dataset(X_val, y_val, sentence_vectorizer, label_vectorizer, tags)
ds_test = generate_dataset(X_test, y_test, sentence_vectorizer, label_vectorizer, tags)

In [25]:
for x, y in ds_train.take(1):
    print(f"X: {x}")
    print(f"Y: {y}")

X: [  723    11   273    14 10143     8  1177  3763   944    23   761     5
    34  3601    16    93  4461  2413    94  1892     6   425  1853     5
    44     3     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
Y: [ 2 16 16 16 16 16 16 16 16 16 16 16  6  5 16 16 16 16 16 16 16 16 16 16
 16 16 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1]


In [26]:
def NER(vocab_size, emb_dim, tags_size):
    model = tf.keras.Sequential()

    # vocab_size +1 -> Why +1? the padding token '0'
    # why mask_zero? The value 0 should be reserved as the mask value, as it will be ignored in training.
    model.add(layers.Embedding(input_dim=vocab_size+1, output_dim=emb_dim, mask_zero=True))
    model.add(layers.LSTM(units=emb_dim, return_sequences=True))
    model.add(layers.Dense(units=tags_size, activation=tf.nn.log_softmax))
    return model

In [27]:
def masked_loss(y_true, y_pred):
   loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, ignore_class=-1)
   loss = loss_fn(y_true, y_pred)
   return  loss

In [28]:
def masked_accuracy(y_true, y_pred):
   

    y_true = tf.cast(y_true, tf.float32)

    mask = tf.not_equal(y_true, -1)
    mask = tf.cast(mask, tf.float32)

    y_pred_class = tf.math.argmax(y_pred, axis=-1)
    y_pred_class = tf.cast(y_pred_class, tf.float32)

    matches_true_pred  = tf.equal(y_true, y_pred_class)
    matches_true_pred = tf.cast(matches_true_pred , tf.float32)

    matches_true_pred *= mask

    masked_acc = tf.reduce_sum(matches_true_pred) / tf.reduce_sum(mask)

    return masked_acc

In [29]:
model = NER(len(vocab), 128, len(tags))

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.01),
    loss=masked_loss,
    metrics=[masked_accuracy]
)

In [30]:
BATCH_SIZE = 64

history = model.fit(
    ds_train.batch(BATCH_SIZE),
    validation_data=ds_val.batch(BATCH_SIZE),
    shuffle=True,
    epochs=2
)

Epoch 1/2
734/734 ━━━━━━━━━━━━━━━━━━━━ 82s 108ms/step - loss: 0.3117 - masked_accuracy: 0.9229 - val_loss: 0.1099 - val_masked_accuracy: 0.9656
Epoch 2/2
734/734 ━━━━━━━━━━━━━━━━━━━━ 76s 103ms/step - loss: 0.0984 - masked_accuracy: 0.9682 - val_loss: 0.1105 - val_masked_accuracy: 0.9651


In [31]:
model.save_weights('NER.weights.h5')

In [32]:
model.load_weights('NER.weights.h5')

In [33]:
test_sentences_ids = sentence_vectorizer(X_test)
y_true = label_vectorizer(y_test, tags)

y_pred = model.predict(test_sentences_ids)

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step


In [34]:
print(f"The model's accuracy in test set is: {masked_accuracy(y_true, y_pred).numpy():.4f}")

The model's accuracy in test set is: 0.9613


In [35]:
def predict(sentence, model, sentence_vectorizer, tags):

    sentence_ids = sentence_vectorizer(sentence)
    sentence_ids = tf.expand_dims(sentence_ids, axis=0)

    output = model.predict(sentence_ids)

    outputs = np.argmax(output, axis=-1)
    outputs = outputs[0]

    pred = [tags[tag_idx] for tag_idx in outputs]

    return pred

In [39]:
sentence = "Mustafa is going to Gaza (Palestine) next Ramadan with Mohamed to kill the Israel Army"

ner = predict(sentence, model, sentence_vectorizer, tags)

table = [[word, tag] for word, tag in zip(sentence.split(' '), ner)]

print(f"\nSentence: {sentence}\n")
print(f"NER Tags: {' '.join(ner)}\n")
print(tabulate.tabulate(table, headers=['Word', 'Tag'], tablefmt='rounded_outline'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

Sentence: Mustafa is going to Gaza (Palestine) next Ramadan with Mohamed to kill the Israel Army

NER Tags: B-per O O O B-geo I-geo O B-tim O B-per O O O B-geo I-org

╭─────────────┬───────╮
│ Word        │ Tag   │
├─────────────┼───────┤
│ Mustafa     │ B-per │
│ is          │ O     │
│ going       │ O     │
│ to          │ O     │
│ Gaza        │ B-geo │
│ (Palestine) │ I-geo │
│ next        │ O     │
│ Ramadan     │ B-tim │
│ with        │ O     │
│ Mohamed     │ B-per │
│ to          │ O     │
│ kill        │ O     │
│ the         │ O     │
│ Israel      │ B-geo │
│ Army        │ I-org │
╰─────────────┴───────╯
